# 필요한 모듈 설치 및 임포트

In [ ]:
# pip install selenium==4.8.2

In [2]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.15.2


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

In [4]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

In [5]:
# 네이버 웹툰 - 토요일
url = 'https://comic.naver.com/webtoon?tab=sat'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [6]:
sunday_webtoon_button = driver.find_element(By.XPATH, '//*[@id="wrap"]/header/div[3]/nav/ul/li[8]/a') # 일요일 웹툰 버튼 클릭
sunday_webtoon_button.click()

In [7]:
driver.back() # 앞의 페이지로 이동

In [8]:
driver.forward() # 뒤의 페이지로 이동

In [9]:
driver.close() # 브라우저 창 닫기

# 네이버 토, 일 웹툰 정보 크롤링

In [10]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(options=chrome_options)

webtoon_list = []
weekend_dic = {0:'토요일', 1:'일요일'}
url = 'https://comic.naver.com/webtoon?tab=sat'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

for weekend in range(2):
    # 페이지를 가장 아래까지 스크롤하는 방법

    interval = 1 # 1초에 한번씩 스크롤 내림

    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    # 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
    # elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

    columns = ['weekend', 'name', 'author', 'rate']         # 웹툰이름, 웹툰작가, 웹툰평점, 정보 크롤링
    values = []                                             # 토요일 웹툰 리스트와 일요일 웹툰 리스트 정보를 담을 빈 리스트 생성
    soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
    data_rows = soup.find_all('li', attrs={'class':'item'}) # 해당 조건에 맞는 정보들을 list로 반환

    for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

        print('{0} {1}번째 웹툰정보 크롤링'.format(weekend_dic[weekend], i+1))
        blank = []
        blank.append(weekend_dic[weekend])

        name = row.find('span', attrs={'class':'ContentTitle__title--e3qXt'})       # 웹툰이름 가져오기
        if name:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
            name = name.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
            blank.append(name)
        else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
            blank.append('Something is wrong')
            print('{}번째 웹툰이름 가져올때 문제발생'.format(i+1))
            continue


        author = row.find('a', attrs={'class':'ContentAuthor__author--CTAAP'})      # 웹툰작가 가져오기
        if author:
            author = author.get_text().strip()
            blank.append(author)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰작가 가져올때 문제발생'.format(i+1))
            continue


        rate = row.find('span', attrs={'class':'Rating__star_area--dFzsb'})        # 웹툰평점 가져오기
        if rate:
            rate = rate.find('span', attrs={'class':'text'}).get_text().strip()    # class가 Rating__star_area--dFzsb인 span 태그 안에 있는 span 태그에서 텍스트 가져오기
            blank.append(rate)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰평점 가져올때 문제발생'.format(i+1))
            continue


#         image_url = row.find('img', attrs={'class':'Poster__image--d9XTI'})     # 웹툰이름 가져오기
#         if image_url:                                                           # 해당 조건에 맞는 정보가 하나라도 있을 경우
#             image_url = image_url['src']                                        # src attribute의 값을 가져온다
#             if image_url.startswith('//'):                                      # image_url이 https: 로 시작하지 않는 경우
#                 image_url = 'https:' + image_url
#             blank.append(image_url)
#         else:
#             blank.append('Something is wrong')
#             print('{}번째 웹툰이미지 가져올때 문제발생'.format(i+1))
#             continue

        values.append(blank)
        print('---------------------------------------------------')

    sunday_webtoon_button = driver.find_element(By.XPATH, '//*[@id="wrap"]/header/div[3]/nav/ul/li[8]/a') # 일요일 웹툰 클릭
    sunday_webtoon_button.click()
    time.sleep(1)
    
    df = pd.DataFrame(values, columns = columns)
    webtoon_list.append(df) # 최종적으로 [[토요일 웹툰 정보 리스트들], [일요을 웹툰 정보 리스트들]] 형태로 반환됨


토요일 1번째 웹툰정보 크롤링
---------------------------------------------------
토요일 2번째 웹툰정보 크롤링
---------------------------------------------------
토요일 3번째 웹툰정보 크롤링
---------------------------------------------------
토요일 4번째 웹툰정보 크롤링
---------------------------------------------------
토요일 5번째 웹툰정보 크롤링
---------------------------------------------------
토요일 6번째 웹툰정보 크롤링
---------------------------------------------------
토요일 7번째 웹툰정보 크롤링
---------------------------------------------------
토요일 8번째 웹툰정보 크롤링
---------------------------------------------------
토요일 9번째 웹툰정보 크롤링
---------------------------------------------------
토요일 10번째 웹툰정보 크롤링
---------------------------------------------------
토요일 11번째 웹툰정보 크롤링
---------------------------------------------------
토요일 12번째 웹툰정보 크롤링
---------------------------------------------------
토요일 13번째 웹툰정보 크롤링
---------------------------------------------------
토요일 14번째 웹툰정보 크롤링
---------------------------------------------------
토요일 15번째 웹툰정보 크롤링
-----------

In [11]:
df_ = pd.concat(webtoon_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,weekend,name,author,rate
0,토요일,개꿈,신송림,9.88
1,토요일,초인의 시대,섭이,9.95
2,토요일,마도전생기,포스스튜디오 / codezero,9.79
3,토요일,욕망일기Deep,김꿀빨 / 누리마루,8.54
4,토요일,귀환했는데 입대 전날이다,해일 / 오코믹 / 황금비둘기,9.71
5,토요일,망나니 소교주로 환생했다,재무 / 전마두 / 대은호,9.86
6,토요일,아홉수 우리들,수박양,9.97
7,토요일,킬러 배드로,김정현 / 임리나,9.95
8,토요일,작전명 순정,꼬까리 / 들덤,9.96
9,토요일,아이 하나,망순,9.80


In [12]:
df_.shape

(188, 4)

# Dataframe을 csv 파일로 저장

In [ ]:
df_.to_csv('네이버웹툰_토일_크롤링.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장